### This notebook shows how to make a Pandas DataFrame from WarpScript GTS

### 1. From a single GTS to a DataFrame

In [1]:
%load_ext warpscript_cellmagic
%alias_magic w warpscript

Created `%%w` as an alias for `%%warpscript`.


We will need pandas and pickle libraries.

In [2]:
import pandas as pd
import pickle as pkl

We first create a random GTS.

In [3]:
%%w -s s
NEWGTS 'randGTS' RENAME 1 10 <% h RAND RAND NaN RAND ADDVALUE %> FOR

Starting connection with 127.0.0.1:25333.
Creating a new WarpScript stack accessible under variable "s".
top: 	<GTS with 10 values>



In order to make a GTS understood by a python interpreter, we store its content in a map of lists and pickle it as a dict.<br/>
The macro `GTStoPickledDict` does this. To load it, you can place the file `macros/GTStoPickledDict.mc2` in the macro<br/>
folder of the Warp10 platform you are sending requests to, or you can execute the following cell.

In [4]:
%%w -s s
<%
    # Documenting the macro
    'GTS BOOLEAN @GTStoPickledDict' DOC

    # Check there is two arguments on the stack
    <% DEPTH 2 < %> <% 'Macro takes two arguments' MSGFAIL %> IFT
        
    # Check that top is a boolean indicating whether to use GTS classname or selector
    <% 1 PICK TYPEOF 'BOOLEAN' != %> <% 'First argument must be a boolean indicating whether to use GTS selector (true) or classname (false)' MSGFAIL %> IFT
    
    # Check that second argument is a GTS
    <% 2 PICK TYPEOF 'GTS' != %> <% 'Second argument must be a GTS' MSGFAIL %> IFT
    
    # Store the arguments
    'withSelector' STORE
    'gts' STORE
    
    # Make name
    $gts <% $withSelector %> <% TOSELECTOR %> <% NAME %> IFTE
    'name' STORE
    
    # macro: check not all NaN (for locations and elevations)
    <% UNIQUE DUP SIZE 1 == SWAP 0 GET ISNaN && %> 'isAllNaN' STORE
        
    # Return pickled dict for pandas
    {
        # ticks
        'timestamps' $gts TICKLIST
        
        # locations
        $gts LOCATIONS 'lon' STORE 'lat' STORE
        <% $lat @isAllNaN ! %> <% $name '.lat' + $lat %> IFT
        <% $lon @isAllNaN ! %> <% $name '.lon' + $lon %> IFT
        
        # elevations
        $gts ELEVATIONS 'elev' STORE
        <% $elev @isAllNaN ! %> <% $name '.elev' + $elev %> IFT
        
        # values        
        $name $gts VALUES
    }
    ->PICKLE
%>
'GTStoPickledDict' STORE

top: 	<GTS with 10 values>



We evaluate the macro on the random GTS that was left on the stack.<br/>
Setting the first argument to false means we drop its labels for its pickled representation.

In [5]:
%%w -s s
false
@GTStoPickledDict

top: 	b'\x80\x02}q\x00(X\n\x00\x00\x00timestampsq\x01]q\x02(I3600000000\nI7200000000\nI10800000000\nI14400000000\nI18000000000\nI21600000000\nI25200000000\nI28800000000\nI32400000000\nI36000000000\neX\x0b\x00\x00\x00randGTS.latq\x03]q\x04(G?\xd1rc(\x00\x00\x00G?\xe2\xb2z\x0c\x00\x00\x00G?\xe3Y/\x89\x80\x00\x00G?\xe3\xcc5\xcc\x00\x00\x00G?\xb5\xf0e\x1c\x00\x00\x00G?\xe4cIF\x80\x00\x00G?\xe6Ry\x00\x00\x00\x00G?\xe2m2(\x80\x00\x00G?\xe7\xcc\xbeF\x80\x00\x00G?\xecJ\x08\\\x80\x00\x00eX\x0b\x00\x00\x00randGTS.lonq\x05]q\x06(G?\xd9Df\x00\x00\x00\x00G?\xda!Bd\x00\x00\x00G?\xe6\xb5\xbcw\x00\x00\x00G?\xef\x1b\x01\xb5\x00\x00\x00G?\xd2\x06\xf6Z\x00\x00\x00G?\xe9\x9b\xed\xe4\x00\x00\x00G?\xdfC\rr\x00\x00\x00G?\xdc\x02\x16\x9e\x00\x00\x00G?\xe6@e\x08\x00\x00\x00G?\xd7,\x82:\x00\x00\x00eX\x07\x00\x00\x00randGTSq\x07]q\x08(G?\xda\x8b\xb3\xbf\xa9\xcf\xbcG?\xd0\xb3\xb9fk\xd8\xaaG?\xdc\xcc\xa4a\xbb\xb5\x8aG?\xe5\xa7\xa9\xdam`nG?\xe1\x96\x8bA\x0b\xac\xafG?\xc9`4\x8a@\xb6\\G?\xcf0\x93>\x99\x18\xa4G?\xc4Wq

We then load the dict from its pickled representation and create a pandas dataframe with it.

In [6]:
gts1 = s.pop()
df1 = pd.DataFrame.from_dict(pkl.loads(gts1))
df1

,timestamps,randGTS.lat,randGTS.lon,randGTS
0,3600000000,0.272607,0.394800,0.414777
1,7200000000,0.584287,0.408280,0.260969
2,10800000000,0.604637,0.709685,0.449990
3,14400000000,0.618678,0.972047,0.676717
4,18000000000,0.085699,0.281675,0.549627
5,21600000000,0.637120,0.800284,0.198248
6,25200000000,0.697567,0.488468,0.243670
7,28800000000,0.575830,0.437627,0.158919
8,32400000000,0.743743,0.695361,0.596695
9,36000000000,0.884037,0.362092,0.306283


In the following example, we choose to keep label information.

In [7]:
%%w -s s
NEWGTS 'randGTS' RENAME 1 10 <% h RAND RAND NaN RAND ADDVALUE %> FOR
{ 'key1' 'info1' 'key2' 'info2' } RELABEL
true
@GTStoPickledDict

top: 	b'\x80\x02}q\x00(X\n\x00\x00\x00timestampsq\x01]q\x02(I3600000000\nI7200000000\nI10800000000\nI14400000000\nI18000000000\nI21600000000\nI25200000000\nI28800000000\nI32400000000\nI36000000000\neX"\x00\x00\x00randGTS{key1=info1,key2=info2}.latq\x03]q\x04(G?\xc7 \x01\x9a\x00\x00\x00G?\xef\xd1-\x87\x00\x00\x00G?\xe8\xda\xf1\xc7\x80\x00\x00G?\xee\xc5|\xae\x00\x00\x00G?\xe3<m\xa4\x80\x00\x00G?\xecu\x83\xf9\x00\x00\x00G?\xe3\xec9\x82\x80\x00\x00G?\xee\xee\xcfu\x80\x00\x00G?\xe1\xd1*\xf2\x00\x00\x00G?\xc7M\xcb\x0c\x00\x00\x00eX"\x00\x00\x00randGTS{key1=info1,key2=info2}.lonq\x05]q\x06(G?\xbdP\x03\x80\x00\x00\x00G?\xce\x9c\xbdx\x00\x00\x00G?\xeaL\x9a\x95\x00\x00\x00G?\xd5\xc4n0\x00\x00\x00G?\x9c"_\x80\x00\x00\x00G?\xbaU\xb3(\x00\x00\x00G?\xe0)\xd0P\x00\x00\x00G?\xd7\xed\x82\x1e\x00\x00\x00G?\xe0\x80^\x98\x00\x00\x00G?\xa5-l`\x00\x00\x00eX\x1e\x00\x00\x00randGTS{key1=info1,key2=info2}q\x07]q\x08(G?\xe3\x05s\xd0\xc9\xb5_G?\xeb\x02\x9cTv\x87\x87G?\xe6x\x9e\xfc\xaa\xb5+G?\xd0.VFY\xef\x06G?\xd

In [8]:
gts2 = s.pop()
df2 = pd.DataFrame.from_dict(pkl.loads(gts2))
df2

,timestamps,"randGTS{key1=info1,key2=info2}.lat","randGTS{key1=info1,key2=info2}.lon","randGTS{key1=info1,key2=info2}"
0,3600000000,0.180664,0.114502,0.594416
1,7200000000,0.994284,0.239158,0.844069
2,10800000000,0.776727,0.821851,0.702224
3,14400000000,0.961607,0.340114,0.252828
4,18000000000,0.601127,0.027475,0.250530
5,21600000000,0.889345,0.102870,0.129245
6,25200000000,0.622586,0.505104,0.247340
7,28800000000,0.966652,0.373871,0.015785
8,32400000000,0.556783,0.515670,0.756590
9,36000000000,0.182062,0.041362,0.218084


We can also not use geo information.

In [9]:
%%w -s s
NEWGTS 'randTS' RENAME 2 11 <% h NaN NaN NaN RAND ADDVALUE %> FOR
false
@GTStoPickledDict

top: 	b'\x80\x02}q\x00(X\n\x00\x00\x00timestampsq\x01]q\x02(I7200000000\nI10800000000\nI14400000000\nI18000000000\nI21600000000\nI25200000000\nI28800000000\nI32400000000\nI36000000000\nI39600000000\neX\x06\x00\x00\x00randTSq\x03]q\x04(G?\xebd\x8c\xb9\xa7F\x1bG?\xd22\xc2R:\x87\xb4G?\xef\xd3\xf2\xdeH*\xcdG?\xee\xa80F\x16\x95)G?\xeb\x85\xdd\x04Gh\xb3G?\xd1Un\x0f\xdd\x13\x0cG?\xaed.y|\xbe\x80G?\xee\\\xb8+%\xe9\x9cG?\xe3\xfe\xf0*~F1G?\xbe\x98\t\x11\xe9\x15\x88eu.'



In [10]:
gts3 = s.pop()
df3 = pd.DataFrame.from_dict(pkl.loads(gts3))
df3

,timestamps,randTS
0,7200000000,0.856024
1,10800000000,0.284348
2,14400000000,0.994623
3,18000000000,0.958031
4,21600000000,0.860091
5,25200000000,0.270839
6,28800000000,0.059358
7,32400000000,0.948818
8,36000000000,0.624870
9,39600000000,0.119507


### 2. Revert a DataFrame to a GTS

To revert a DataFrame to a GTS, we first need to convert the DataFrame into a dict.

In [11]:
gts1b = df1.to_dict('list')
gts1b

{'timestamps': [3600000000,
  7200000000,
  10800000000,
  14400000000,
  18000000000,
  21600000000,
  25200000000,
  28800000000,
  32400000000,
  36000000000],
 'randGTS.lat': [0.27260664850473404,
  0.5842867121100426,
  0.6046369252726436,
  0.6186779960989952,
  0.08569938596338034,
  0.6371199013665318,
  0.6975674629211426,
  0.5758295813575387,
  0.7437430741265416,
  0.8840371901169419],
 'randGTS.lon': [0.39479970932006836,
  0.40827998891472816,
  0.709684593603015,
  0.9720467124134302,
  0.2816749457269907,
  0.8002843335270882,
  0.4884675610810518,
  0.437627462670207,
  0.6953606754541397,
  0.36209159530699253],
 'randGTS': [0.41477674214479143,
  0.2609694957931291,
  0.44999036354314426,
  0.6767167345196989,
  0.5496269483968684,
  0.19824845076202535,
  0.24366989666504424,
  0.1589185385000157,
  0.5966953663182542,
  0.30628282751016445]}

We can push this dict directly onto the stack, since it will be automatically converted in the JVM.

In [12]:
s.push(gts1b)
s

top: 	{'randGTS.lat': [0.27260664850473404, 0.5842867121100426, 0.6046369252726436, 0.6186779960989952, 0.08569938596338034, 0.6371199013665318, 0.6975674629211426, 0.5758295813575387, 0.7437430741265416, 0.8840371901169419], 'timestamps': [3600000000, 7200000000, 10800000000, 14400000000, 18000000000, 21600000000, 25200000000, 28800000000, 32400000000, 36000000000], 'randGTS': [0.41477674214479143, 0.2609694957931291, 0.44999036354314426, 0.6767167345196989, 0.5496269483968684, 0.19824845076202535, 0.24366989666504424, 0.1589185385000157, 0.5966953663182542, 0.30628282751016445], 'randGTS.lon': [0.39479970932006836, 0.40827998891472816, 0.709684593603015, 0.9720467124134302, 0.2816749457269907, 0.8002843335270882, 0.4884675610810518, 0.437627462670207, 0.6953606754541397, 0.36209159530699253]}

Now we can use the lists contained in this map to populate a GTS.

In [13]:
%%w -s s
'dict' STORE
$dict 'timestamps' GET
$dict 'randGTS.lat' GET
$dict 'randGTS.lon' GET
[] // no elevation
$dict 'randGTS' GET
MAKEGTS 'randGTS' RENAME

top: 	<GTS with 10 values>



In [14]:
print(s.pop().toString())

randGTS{}
=3600000000/0.27260664850473404:0.39479970932006836/ 0.41477674214479143
=7200000000/0.5842867121100426:0.40827998891472816/ 0.2609694957931291
=10800000000/0.6046369252726436:0.709684593603015/ 0.44999036354314426
=14400000000/0.6186779960989952:0.9720467124134302/ 0.6767167345196989
=18000000000/0.08569938596338034:0.2816749457269907/ 0.5496269483968684
=21600000000/0.6371199013665318:0.8002843335270882/ 0.19824845076202535
=25200000000/0.6975674629211426:0.4884675610810518/ 0.24366989666504424
=28800000000/0.5758295813575387:0.437627462670207/ 0.1589185385000157
=32400000000/0.7437430741265416:0.6953606754541397/ 0.5966953663182542
=36000000000/0.8840371901169419:0.36209159530699253/ 0.30628282751016445



### 3. From a list of GTS to a DataFrame

We want to put every GTS of a list in a same DataFrame with a single `timestamps` column.<br/>
Since every GTS don't have values for the same timestamps, we need to handle missing values,<br/>
and we need to make the assumption that each GTS can have at most one value per timestamp.<br/>
It is more efficient to do that in WarpScript, as done in by the macro `ListGTStoPickledDict`.

In [15]:
%%w -s s -o
<%
    # Documenting the macro
    '[GTS] BOOLEAN @ListGTStoPickledDict' DOC

    # Check there is two arguments on the stack
    <% DEPTH 2 < %> <% 'Macro takes two arguments' MSGFAIL %> IFT
        
    # Check that top is a boolean indicating whether to use GTS classname or selector
    <% 1 PICK TYPEOF 'BOOLEAN' != %> <% 'First argument must be a boolean indicating whether to use GTS selector (true) or classname (false)' MSGFAIL %> IFT
    
    # Check that second argument is a list of GTS
    <% 2 PICK TYPEOF 'LIST' != %> <% 'Second argument must be a List of GTS' MSGFAIL %> IFT
    2 PICK <% <% TYPEOF 'GTS' != %> <% 'Second argument is a list that has an element that is not a GTS' MSGFAIL %> IFT %> FOREACH
    
    # Store the arguments
    'withSelector' STORE
    'gtsList' STORE
    
    # make tickbase of all GTS
    $gtsList TICKS 'ticks' STORE
    $ticks [] [] [] $ticks MAKEGTS 'baseGTS' STORE
    
    # macro: check not all NaN (for locations and elevations)
    <% UNIQUE DUP SIZE 1 == SWAP 0 GET ISNaN && %> 'isAllNaN' STORE
        
    # Return pickled dict for pandas
    {
        # ticks
        'timestamps' $ticks
        
        # loop over list of GTS
        $gtsList
        <%
            'gts' STORE
            
            # Make name
            $gts <% $withSelector %> <% TOSELECTOR %> <% NAME %> IFTE
            'name' STORE
        
            # Put on the same tick base and fill missing values with NaN
            [ $gts true mapper.replace 0 0 0 ] MAP
            'mask' STORE
            [ $mask [ $baseGTS ] [] op.negmask ] APPLY
            [ SWAP NaN mapper.replace 0 0 0 ] MAP
            0 GET 'residualSeries' STORE
            [ $gts $residualSeries ] MERGE SORT
            'gts' STORE
        
            # locations
            $gts LOCATIONS 'lon' STORE 'lat' STORE
            <% $lat @isAllNaN ! %> <% $name '.lat' + $lat %> IFT
            <% $lon @isAllNaN ! %> <% $name '.lon' + $lon %> IFT
        
            # elevations
            $gts ELEVATIONS 'elev' STORE
            <% $elev @isAllNaN ! %> <% $name '.elev' + $elev %> IFT
        
            # values        
            $name $gts VALUES
        %>
        FOREACH
    }
    ->PICKLE
%>
'ListGTStoPickledDict' STORE

Creating a new WarpScript stack accessible under variable "s".



We apply the macro `ListGTStoPickledDict` similarly than `GTStoPickledDict`,<br/>
except that it takes a list of GTS instead of a single GTS as second argument.

In [16]:
%%w -s s
[ NEWGTS 'randGTS' RENAME 1 10 <% h RAND RAND NaN RAND ADDVALUE %> FOR
  NEWGTS 'randTS' RENAME 2 11 <% h NaN NaN NaN RAND ADDVALUE %> FOR
  NEWGTS 'stringTS' RENAME 5 8 <% h NaN NaN NaN 'a string' ADDVALUE %> FOR ]
false
@ListGTStoPickledDict

top: 	b'\x80\x02}q\x00(X\n\x00\x00\x00timestampsq\x01]q\x02(I3600000000\nI7200000000\nI10800000000\nI14400000000\nI18000000000\nI21600000000\nI25200000000\nI28800000000\nI32400000000\nI36000000000\nI39600000000\neX\x0b\x00\x00\x00randGTS.latq\x03]q\x04(G?\xae\x9b\xee\xc0\x00\x00\x00G?\xbe\x08I\x90\x00\x00\x00G?\xed\x04\x9b\xa4\x80\x00\x00G?\xbf\xb7_\xd8\x00\x00\x00G?\xe5\x94h\xd6\x80\x00\x00G?\xd5a\xd0O\x00\x00\x00G?\x8cS\xcf\xc0\x00\x00\x00G?\xe5\xb0L\xe5\x80\x00\x00G?\xc9L\xa0\xea\x00\x00\x00G?\xef\x12\xcf\xd8\x80\x00\x00G\x7f\xf8\x00\x00\x00\x00\x00\x00eX\x0b\x00\x00\x00randGTS.lonq\x05]q\x06(G?\xeb~jG\x00\x00\x00G?\xd1Pn\x12\x00\x00\x00G?\xc48\xff,\x00\x00\x00G?\xec\x07sK\x00\x00\x00G?\xc3\xbd\x0b\xd8\x00\x00\x00G?\xebk\xb3/\x00\x00\x00G?\xe3\xb8\x1cT\x00\x00\x00G?\xd7[\xab\xf0\x00\x00\x00G?\xb6I4\xa8\x00\x00\x00G?\xcc \x9e4\x00\x00\x00G\x7f\xf8\x00\x00\x00\x00\x00\x00eX\x07\x00\x00\x00randGTSq\x07]q\x08(G?\xd5\xb1 \x1c\x18\x82DG?\xe9\xc4\x9f7\xa2\xf1\xe6G?\xb8\xd44\x05\x12\xfb G?\

Contrary to our first example with a single GTS, the following cell will raise<br/>
an error if a GTS of the list has a timestamp with multiple values.

In [17]:
listGts = s.pop()
df4 = pd.DataFrame.from_dict(pkl.loads(listGts))
df4

,timestamps,randGTS.lat,randGTS.lon,randGTS,randTS,stringTS
0,3600000000,0.059783,0.859182,0.338936,NaN,NaN
1,7200000000,0.117314,0.270534,0.805252,0.167504,NaN
2,10800000000,0.906812,0.157989,0.096988,0.123669,NaN
3,14400000000,0.123892,0.875909,0.639887,0.021203,NaN
4,18000000000,0.674366,0.154207,0.672360,0.050268,a string
5,21600000000,0.334095,0.856897,0.531092,0.320759,a string
6,25200000000,0.013832,0.616224,0.759840,0.047716,a string
7,28800000000,0.677771,0.364970,0.044925,0.667506,a string
8,32400000000,0.197651,0.087055,0.999778,0.967449,NaN
9,36000000000,0.971046,0.219745,0.591541,0.463399,NaN
